In [ ]:
  !pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 3.5 MB/s eta 0:00:00


In [ ]:
pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.8 MB/s eta 0:00:00


In [ ]:
pip install fuzzywuzzy python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 25.8 MB/s eta 0:00:00


In [ ]:
pip install indonlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 2.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import string
import re
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
#import emoji
from sklearn.utils import resample
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from sklearn.utils.class_weight import compute_class_weight
import requests
from collections import Counter


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Download and load slang word lists
slang_urls = [
    "https://raw.githubusercontent.com/ramaprakoso/analisis-sentimen/master/kamus/kbba.txt",
    "https://raw.githubusercontent.com/agusmakmun/SentiStrengthID/master/id_dict/slangword.txt",
    "https://raw.githubusercontent.com/panggi/pujangga/master/resource/formalization/formalizationDict.txt"
]

slang_words = set()
for url in slang_urls:
    response = requests.get(url)
    slang_words.update(response.text.splitlines())


# Download and load stopwords lists
stopword_urls = [
    "https://raw.githubusercontent.com/yasirutomo/python-sentianalysis-id/master/data/feature_list/stopwordsID.txt",
    "https://raw.githubusercontent.com/ramaprakoso/analisis-sentimen/master/kamus/stopword.txt",
    "https://raw.githubusercontent.com/abhimantramb/elang/master/word2vec/utils/stopwords-list/stopwordsID.txt"
]

stop_words = set()
for url in stopword_urls:
    response = requests.get(url)
    stop_words.update(response.text.splitlines())


In [ ]:
# Function to remove slang words
def remove_slang_words(text, slang_words):
    words = word_tokenize(text.lower())
    cleaned_words = [word for word in words if word not in slang_words]
    return ' '.join(cleaned_words)

# Function to remove stopwords
def remove_stopwords(text, stop_words):
    words = word_tokenize(text.lower())
    cleaned_words = [word for word in words if word not in stop_words]
    return ' '.join(cleaned_words)

In [ ]:
data['text'] = data['text'].apply(lambda x: remove_slang_words(x, slang_words))
data['text'] = data['text'].apply(lambda x: remove_stopwords(x, stop_words))

In [ ]:
data.tail()

,text,label
4340,berencana pindah ikn rezeki semoga anak cucu j...,Geografi
4341,kuliah gratis berdampak positif anak kota pelo...,Geografi
4342,dampak kuliah gratis dirasakan anak kota pelos...,Geografi
4343,kuliah gratis berdampak positif anak kota pelo...,Geografi
4344,dampak kuliah gratis anak tinggal kota pelosok...,Geografi


In [ ]:
data.tail()

,text,label
4340,berencana pindah ikn rezeki semoga anak cucu j...,Geografi
4341,kuliah gratis berdampak positif anak kota pelo...,Geografi
4342,dampak kuliah gratis dirasakan anak kota pelos...,Geografi
4343,kuliah gratis berdampak positif anak kota pelo...,Geografi
4344,dampak kuliah gratis anak tinggal kota pelosok...,Geografi


# PROSES CLEANING

In [ ]:
# Function to clean data
def clean_tweet(tweet):
    import re

    # Define regex pattern to match usernames, hyperlinks, and old style retweets
    pattern = r'(@[\w/+=]+)|(https?://\S+)|(\[RE\s+\w+\])'

    # Remove usernames, hyperlinks, and old style retweets
    tweet = re.sub(pattern, '', tweet)

    # Remove numbers
    tweet = re.sub('[0-9]+', '', tweet)

    # Remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)

    # Menghapus teks retweet gaya lama "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)

    # Remove special characters using regular expressions (re) hashtag
    tweet = re.sub(r'[^\w\s]', '', tweet)

    return tweet

# Apply the cleaning function to every row in the 'text' column of the DataFrame
df['text'] = df['text'].apply(clean_tweet)


In [ ]:
#import stopword
from nltk.corpus import stopwords
stopwords_indonesia = stopwords.words('indonesian')

#import sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#tokenize
from nltk.tokenize import TweetTokenizer

# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';(', ':\U0001F92D'
    ])

# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)

In [ ]:
# Remove emoji
def remove_emojis(tweet):
    """
    Removes emojis from text while preserving punctuation, special characters,
    and characters from various languages (Korean, Chinese, Russian, German, Spanish, etc.).

    Args:
        tweet (str): The text to remove emojis from.

    Returns:
        str: The text with emojis removed.
    """

    # Unicode emoji character ranges (excluding flags)
    emoji_ranges = [
        u"\U0001F600-\U0001F64F",  # emoticons
        u"\U0001F300-\U0001F5FF",  # symbols & pictographs
        u"\U0001F680-\U0001F6FF",  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"   # miscellaneous symbols
    ]

    # Combine emoji ranges into a single pattern
    emoji_pattern = "|".join(emoji_ranges)

    # Create a pattern that matches any character except letters, numbers, punctuation,
    # whitespace, and characters from various languages (excluding flags)
    allowed_chars = r"[a-zA-Z0-9\s\-.,!?:;\(\)\[\]\{\}]"
    non_emoji_pattern = f"{allowed_chars}"

    # Combine the emoji and non-emoji patterns into a single pattern
    combined_pattern = f"(^{non_emoji_pattern})|({emoji_pattern})"

    # Remove emojis while keeping other characters
    return re.sub(combined_pattern, r"\1", tweet, flags=re.UNICODE)

df['text'] = df['text'].apply(remove_emojis)

In [ ]:
#Remove incorrect emoji
def incorrect_emoji(tweet):
    # Define patterns for incorrect encodings and non-standard characters
    encoding_pattern = re.compile(r"Ã[A-Za-z0-9ÂŒÂˆÂŠÂœ]+")

    # Remove these patterns from the text
    tweet = re.sub(encoding_pattern, '', tweet)

    # Remove any remaining non-standard characters
    tweet = re.sub(r'[^\x00-\x7F]+', '', tweet)

    return tweet

df['text'] = df['text'].apply(incorrect_emoji)

In [ ]:
#remove duplicate with same label
df = df.drop_duplicates(subset=['text', 'label'], keep='first')

In [ ]:
#remove duplicate with different label
df = df.drop_duplicates(subset=['text'], keep= False)

In [ ]:
from indoNLP.preprocessing import replace_slang

# Apply replace_slang to the 'text' column
df['text'] = df['text'].apply(lambda x: replace_slang(str(x)))

# Display the first few rows of the DataFrame to verify the changes
df.head()


,text,label
0,Kunjungan Prabowo ini untuk meresmikan dan men...,Sumber Daya Alam
1,Anies dapat tepuk tangan meriah saat jadi Rekt...,Politik
2,memang benar sih pendukung ada yang goblok b...,Demografi
3,Sewaktu anies bersikap kritis ke kinerja pak p...,Politik
4,Anies Baswedan Harap ASN termasuk TNI dan Polr...,Politik


In [ ]:
from indoNLP.preprocessing import replace_word_elongation

# Apply replace_word_elongation function to the 'text' column of your DataFrame
df['text'] = df['text'].apply(replace_word_elongation)


In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Download stopwords for the Indonesian language
nltk.download('stopwords')
nltk.download('punkt')

# Tokenize the text
df['text'] = df['text'].apply(lambda x: word_tokenize(str(x).lower()))  # Convert to lowercase

# Remove stopwords from tokens
stop_words = set(stopwords.words('indonesian'))
df['text'] = df['text'].apply(lambda x: [word for word in x if word not in stop_words])

# Function to remove stopwords from list of words
def remove_stopwords(words):
    factory = StopWordRemoverFactory()
    stopword_remover = factory.create_stop_word_remover()
    cleaned_words = stopword_remover.remove(' '.join(words)).split()
    return cleaned_words

# Apply the remove_stopwords function to each list of words in the 'text' column
df['text'] = df['text'].apply(remove_stopwords)

# Join the list of words back into a string
df['text'] = df['text'].apply(' '.join)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Tokenize the text
df['text'] = df['text'].apply(lambda x: word_tokenize(str(x).lower()))  # Convert to lowercase

# Remove stopwords from tokens
stop_words = set(stopwords.words('indonesian'))
df['text'] = df['text'].apply(lambda x: [word for word in x if word not in stop_words])


In [ ]:
#find the most use word

# Tokenize the text and count word frequencies
all_words = ' '.join([' '.join(text) for text in df['text']]).lower()
tokens = word_tokenize(all_words)
word_freq = Counter(tokens)

# Display the most common words
most_common_words = word_freq.most_common(10)  # Change 10 to the desired number of most common words
print("Most common words:")
for word, frequency in most_common_words:
    print(word, frequency)

Most common words:
anies 1880
ganjar 1760
prabowo 1561
pranowo 849
mahfud 813
indonesia 622
jnk 598
ganjarmahfud 596
ganjarpranowopilihanumat 558
capres 543


In [ ]:
#Remove kata-kata yang belum terdeteksi

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# List of words to remove
words_to_remove = [
    'ganjarmahfudrebound', 'md', 'baswedan', 'gibran', 'prabowogibran', 'orang',
    'subianto', 'amin', 'ga', 'no', 'amp', 'gm', 'nya', 'imin', 'aniesmuhaimin', 'prof', 'utk', 'lbihbaik',
    'beliau', 'abah', 'mas', 'pak', 'aja', 'kalo', 'presiden', 'gak', 'lu', 'gw', 'gue', 'lu', 'sm', 'w', 'doang', 'mah', 'ku', 'goblok',
    'retweet', 'reply', 'sih', 'dg', 'sj', 'jg'
]

# Add words from most_common_words to words_to_remove
for word, _ in most_common_words:
    words_to_remove.append(word)

def remove_specific_words(text):
    cleaned_words = []
    for word in text:
        # Check if the word is in the list of words to remove
        if word.lower() in words_to_remove:
            continue
        cleaned_words.append(word)
    return cleaned_words


In [ ]:

# Remove specific words from tokens
df['text'] = df['text'].apply(remove_specific_words)

# Join the list of words back into a string (if needed)
df['text'] = df['text'].apply(lambda x: ' '.join(x))


In [ ]:
df.head()

,text,label
0,kunjungan meresmikan menyerahkan proyek bantua...,Sumber Daya Alam
1,tepuk tangan meriah rektor mewajibkan mata kul...,Politik
2,pendukung pendukung ridwan kamil skemanya terb...,Demografi
3,bersikap kritis kinerja dianggap sopan diangga...,Politik
4,harap asn tni polri pegang sumpahnya pemilu,Politik


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# INDOBERT PERCOBAAN 0.4

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import string
import re
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
#import emoji
from sklearn.utils import resample
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from sklearn.utils.class_weight import compute_class_weight



In [ ]:
#read data
data = pd.read_csv('/content/df (1).csv')

In [ ]:
data = data.dropna(subset=['text'])

In [ ]:
df_demografi = data[data['label']=='Demografi']
print("df_demografi.shape = ", df_demografi.shape)

df_ekonomi = data[data['label']=='Ekonomi']
print("df_ekonomi.shape = ", df_ekonomi.shape)

df_geografi = data[data['label']=='Geografi']
print("df_geografi.shape = ", df_geografi.shape)

df_ideologi = data[data['label']=='Ideologi']
print("df_ideologi.shape = ", df_ideologi.shape)

df_hankam = data[data['label']=='Pertahanan dan Keamanan']
print("df_hankam.shape = ", df_hankam.shape)

df_politik = data[data['label']=='Politik']
print("df_politik.shape = ", df_politik.shape)

df_sosbud = data[data['label']=='Sosial Budaya']
print("df_sosbud.shape = ", df_sosbud.shape)

df_sda = data[data['label']=='Sumber Daya Alam']
print("df_sda.shape = ", df_sda.shape)

df_demografi.shape =  (99, 2)
df_ekonomi.shape =  (268, 2)
df_geografi.shape =  (89, 2)
df_ideologi.shape =  (253, 2)
df_hankam.shape =  (257, 2)
df_politik.shape =  (2860, 2)
df_sosbud.shape =  (384, 2)
df_sda.shape =  (134, 2)


In [ ]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler


# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Encode labels
label_encoder = LabelEncoder()
data['encoded_label'] = label_encoder.fit_transform(data['label'])

# # Calculate class weights
# class_weights = compute_class_weight('balanced', classes=np.unique(data['encoded_label']), y=data['encoded_label'])
# class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# # Tokenize text
# tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1', do_lower_case=True)
# data['input_ids'] = data['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=512))

# Split the data
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data['text'].tolist(), data['encoded_label'], test_size=0.2, stratify=data['encoded_label'], random_state=42)

# Convert to DataFrame for compatibility with RandomOverSampler
train_df = pd.DataFrame({'text': train_texts, 'label': train_labels})

# # Apply Random Oversampling
# ros = RandomOverSampler(random_state=42)
# X_resampled, y_resampled = ros.fit_resample(train_df[['text']], train_df['label'])

# # Convert back to list
# train_texts_resampled = X_resampled['text'].tolist()
# train_labels_resampled = y_resampled.tolist()

In [ ]:
# Apply Random Undersampling to reduce the majority class
rus = RandomUnderSampler(random_state=42)
X_undersampled, y_undersampled = rus.fit_resample(train_df[['text']], train_df['label'])

# Convert undersampled data to DataFrame
undersampled_df = pd.DataFrame({'text': X_undersampled['text'], 'label': y_undersampled})

# Apply Random Oversampling to increase the minority class
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(undersampled_df[['text']], undersampled_df['label'])

# Convert back to list
train_texts_resampled = X_resampled['text'].tolist()
train_labels_resampled = y_resampled.tolist()

# Verify resampling results
from collections import Counter

# Check the distribution of the resampled labels
print("Distribution of resampled labels:", Counter(train_labels_resampled))

Distribution of resampled labels: Counter({0: 71, 1: 71, 2: 71, 3: 71, 4: 71, 5: 71, 6: 71, 7: 71})


In [ ]:
# tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1', do_lower_case=True)


from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("indolem/indobertweet-base-uncased", do_lower_case=True)
model = AutoModel.from_pretrained("indolem/indobertweet-base-uncased")


In [ ]:
# Define maximum length for sequences
max_length = 128

# Prepare the data for the DataLoader
def tokenize_function(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt')

train_encodings = tokenize_function(train_texts_resampled)
test_encodings = tokenize_function(test_texts)

# # Tokenize the resampled texts
# train_encodings = tokenizer(train_texts_resampled, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
# test_encodings = tokenizer(test_texts, padding=True, truncation=True, max_length=max_length, return_tensors="pt")


In [ ]:
test_labels = test_labels.tolist()


AttributeError: 'list' object has no attribute 'tolist'

In [ ]:
from torch.utils.data import TensorDataset

# Create TensorDatasets
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels_resampled))
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(test_labels))



In [ ]:
# Create DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load the model
# model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', num_labels=len(label_encoder.classes_))


In [ ]:
from torch.optim import AdamW

# # Define optimizer and loss function
# optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=2e-5)
# criterion = torch.nn.CrossEntropyLoss(weight=class_weights)


# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(31923, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
data

,text,label,encoded_label
0,kunjungan meresmikan menyerahkan proyek bantua...,Sumber Daya Alam,7
1,tepuk tangan meriah rektor mewajibkan mata kul...,Politik,5
2,pendukung pendukung ridwan kamil skemanya terb...,Demografi,0
3,bersikap kritis kinerja dianggap sopan diangga...,Politik,5
4,harap asn tni polri pegang sumpahnya pemilu,Politik,5
...,...,...,...
4340,berencana pindah ikn rezeki semoga anak cucu j...,Geografi,2
4341,kuliah gratis berdampak positif anak kota pelo...,Geografi,2
4342,dampak kuliah gratis dirasakan anak kota pelos...,Geografi,2
4343,kuliah gratis berdampak positif anak kota pelo...,Geografi,2


In [ ]:
from transformers import BertForSequenceClassification

# Define the criterion
criterion = torch.nn.CrossEntropyLoss()

# Training and evaluation functions
def evaluate(model, data_loader, criterion, device):
    model.eval()
    val_loss = 0
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids, attention_mask, labels = [item.to(device) for item in batch]

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits  # Correct attribute for logits
            loss = criterion(logits, labels)
            val_loss += loss.item()

            preds = torch.argmax(logits, dim=1).cpu().numpy()
            val_preds.extend(preds)
            val_labels.extend(labels.cpu().numpy())

    balanced_acc = compute_balanced_accuracy(val_labels, val_preds)
    avg_val_loss = val_loss / len(data_loader)
    return avg_val_loss, balanced_acc

# Training loop
best_balanced_accuracy = 0
best_epoch = 0
num_epochs = 10
train_losses = []
val_losses = []
val_balanced_accuracies = []

for epoch in range(num_epochs):
    model.train()
    train_loss = 0

    for batch in train_loader:
        input_ids, attention_mask, labels = [item.to(device) for item in batch]

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # Correct attribute for logits
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)
    val_loss, balanced_acc = evaluate(model, test_loader, criterion, device)

    train_losses.append(avg_train_loss)
    val_losses.append(val_loss)
    val_balanced_accuracies.append(balanced_acc)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {avg_train_loss:.4f}, Validation Loss: {val_loss:.4f}, Balanced Accuracy: {balanced_acc:.4f}")

    # Save the best model
    if balanced_acc > best_balanced_accuracy:
        best_balanced_accuracy = balanced_acc
        best_epoch = epoch + 1
        torch.save(model.state_dict(), 'best_model.pth')

print(f"Best Epoch: {best_epoch}, Best Balanced Accuracy: {best_balanced_accuracy:.4f}")


AttributeError: 'BaseModelOutputWithPoolingAndCrossAttentions' object has no attribute 'logits'

In [ ]:
# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Calculate accuracy
            _, predicted = torch.max(logits, dim=1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

        accuracy = balanced_accuracy_score(all_labels, all_predictions)
        print(f'Epoch {epoch + 1}/{epochs}, Validation Balanced Accuracy: {accuracy:.4f}')

TypeError: BertModel.forward() got an unexpected keyword argument 'labels'

In [ ]:
# Save the model at epoch 3
epoch_to_save = 3
model_dir = './model_checkpoint_epoch_3'
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)


('./model_checkpoint_epoch_3/tokenizer_config.json',
 './model_checkpoint_epoch_3/special_tokens_map.json',
 './model_checkpoint_epoch_3/vocab.txt',
 './model_checkpoint_epoch_3/added_tokens.json')

In [ ]:
# Load the model and tokenizer
from transformers import BertTokenizer, BertForSequenceClassification

model_dir = './model_checkpoint_epoch_3'
model = BertForSequenceClassification.from_pretrained(model_dir)
tokenizer = BertTokenizer.from_pretrained(model_dir)

model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

UNLABELED_DATA

In [ ]:
df = pd.read_csv('/content/dataset_unlabeled_penyisihan_bdc_2024.csv', sep=";")

In [ ]:
df.head()

,IDText,Text
0,TXT0001,Lu mau org prodemokrasi di negara ini bisa pun...
1,TXT0002,Prabowo ditanya soal hutang luar negeri dia me...
2,TXT0003,kiki_daliyo Ganjar Pranowo itulah beliau soso...
3,TXT0004,Prabowo Gibran yang bisa melakukan itu semua ...
4,TXT0005,Lah justru yg gak nyambung junjungan elu ao...


In [ ]:
new_texts = df['Text'].tolist()


In [ ]:
new_encodings = tokenizer(new_texts, padding=True, truncation=True, max_length=max_length, return_tensors="pt")


In [ ]:
# Create a DataLoader for the new data
new_dataset = TensorDataset(new_encodings['input_ids'], new_encodings['attention_mask'])
new_loader = DataLoader(new_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
# Predict labels for new data
model.eval()
all_predictions = []
with torch.no_grad():
    for batch in new_loader:
        input_ids, attention_mask = batch
        input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Get predictions
        _, predicted = torch.max(logits, dim=1)
        all_predictions.extend(predicted.cpu().numpy())

In [ ]:
# Map numerical labels back to class names
predicted_labels = label_encoder.inverse_transform(all_predictions)

# Add predictions to the new_data DataFrame
df['predicted_label'] = predicted_labels

In [ ]:
subset = df[['IDText', 'Text', 'predicted_label']]
subset.to_csv('subset.csv', index=False)

from google.colab import files
files.download('subset.csv')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
tidak_ada_text = df[['IDText', 'predicted_label']]
tidak_ada_text.to_csv('tidak_ada_text.csv', index=False)

from google.colab import files
files.download('tidak_ada_text.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df

,IDText,Text,predicted_label
0,TXT0001,Lu mau org2 pro-demokrasi di negara ini bisa p...,Politik
1,TXT0002,Prabowo ditanya soal hutang luar negeri dia me...,Politik
2,TXT0003,kiki_daliyo Ganjar Pranowo itulah beliau soso...,Ideologi
3,TXT0004,@kumparan Prabowo Gibran yang bisa melakukan i...,Politik
4,TXT0005,@sniperruben45 @uda_zulhendra @ainunnajib Lah ...,Politik
...,...,...,...
995,TXT0996,"Bikin bangga deh, Ganjar-Mahfud mau alokasikan...",Ekonomi
996,TXT0997,Pak Jokowi sebelum pilpres 2024 berbesar hati ...,Politik
997,TXT0998,@datuakrajoangek Sbaiknya si gemot nga usah ik...,Politik
998,TXT0999,kebiasaan merembuk atau bermusyawarah jadi gay...,Politik


# INDOBERT PERCOBAAN YANG 0.7

In [ ]:
dt = pd.read_csv('/content/data (2).csv')

In [ ]:
dt = dt.dropna(subset=['text'])

In [ ]:
dt.head()

,text,label
0,kunjungan meresmikan menyerahkan proyek bantua...,Sumber Daya Alam
1,tepuk tangan meriah rektor mewajibkan mata kul...,Politik
2,pendukung pendukung ridwan kamil skemanya terb...,Demografi
3,bersikap kritis kinerja dianggap sopan diangga...,Politik
4,harap asn tni polri pegang sumpahnya pemilu,Politik


In [ ]:
# Encode labels
label_encoder = LabelEncoder()
dt['encoded_label'] = label_encoder.fit_transform(dt['label'])

In [ ]:
# Split the data into training and testing sets
train_dt, test_dt = train_test_split(dt, test_size=0.8, stratify=dt['label'], random_state=42)

# Balance the training data if it's imbalanced
labels = train_dt['label'].value_counts()
max_count = labels.max()

balanced_train_dts = []

for label in labels.index:
    label_dt = train_dt[train_dt['label'] == label]
    if len(label_dt) < max_count:
        label_dt = resample(label_dt, replace=True, n_samples=max_count, random_state=42)
    balanced_train_dts.append(label_dt)

balanced_train_dt = pd.concat(balanced_train_dts)

# Define labels
labels_list = ["Ideologi", "Politik", "Ekonomi", "Sosial Budaya", "Pertahanan dan Keamanan", "Sumber Daya Alam", "Geografi", "Demografi"]

# Extract text data as a list of strings
train_texts = balanced_train_dt['text'].astype(str).tolist()
test_texts = test_dt['text'].astype(str).tolist()

# # Label encoding
# label_encoder = LabelEncoder()
# balanced_train_dt['label'] = label_encoder.fit_transform(balanced_train_dt['label'])
# test_dt['label'] = label_encoder.transform(test_dt['label'])

# # Ensure that the test set respects the imbalances found in the train set
# balanced_train_dt = train_dt.groupby('label').apply(lambda x: x.sample(frac=1)).reset_index(drop=True)

# # Define labels
# labels = ["Ideologi", "Politik", "Ekonomi", "Sosial Budaya", "Pertahanan dan Keamanan", "Sumber Daya Alam", "Geografi", "Demografi"]

# # Extract text data as a list of strings
# train_texts = balanced_train_dt['text'].tolist()
# test_texts = test_dt['text'].tolist()

from transformers import BertTokenizer, BertForSequenceClassification

# Load IndoBERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1', do_lower_case=True)
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', num_labels=len(labels))



tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tokenize the text data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')

# Convert labels to tensors
train_labels = torch.tensor(balanced_train_dt['encoded_label'].tolist())
test_labels = torch.tensor(test_dt['encoded_label'].tolist())

# Create TensorDatasets
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

In [ ]:
from transformers import BertForSequenceClassification
import torch.optim as optim

In [ ]:
# Create DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define optimizer and loss function
# optimizer = AdamW(model.parameters(), lr=2e-5)
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

criterion = torch.nn.CrossEntropyLoss()

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
from transformers import BertForSequenceClassification, AdamW
import torch.optim as optim

# Load model architecture (already done)
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', num_labels=len(labels))

# # Define optimizer and learning rate
# optimizer = AdamW(model.parameters(), lr=2e-5)
# Define optimizer and learning rate using PyTorch implementation of AdamW
optimizer = optim.AdamW(model.parameters(), lr=2e-5)


# Define loss function (CrossEntropyLoss for classification)
criterion = torch.nn.CrossEntropyLoss()

# Define device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training loop
epochs = 10  # Example, adjust as needed
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        total_correct = 0
        total_count = 0
        for batch in test_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Calculate accuracy
            _, predicted = torch.max(logits, dim=1)
            total_correct += (predicted == labels).sum().item()
            total_count += labels.size(0)

        accuracy = total_correct / total_count
        print(f'Epoch {epoch + 1}/{epochs}, Validation Accuracy: {accuracy:.4f}')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Save the model at epoch 9
epoch_to_save = 9
model_dir = './model_checkpoint_epoch_9'
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

('./model_checkpoint_epoch_9/tokenizer_config.json',
 './model_checkpoint_epoch_9/special_tokens_map.json',
 './model_checkpoint_epoch_9/vocab.txt',
 './model_checkpoint_epoch_9/added_tokens.json')

In [ ]:
from sklearn.metrics import f1_score
#Validation
model.eval()
predictions, true_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()

        # Store predictions and true labels
        predictions.extend(np.argmax(logits, axis=1).flatten())
        true_labels.extend(label_ids.flatten())

# Calculate F1 score
f1 = f1_score(true_labels, predictions, average='weighted')
print(f'F1 Score: {f1:.4f}')

F1 Score: 0.7026


UNLABELED DATA

In [ ]:
df = pd.read_csv('/content/dataset_unlabeled_penyisihan_bdc_2024.csv', sep=";")

In [ ]:
new_texts = df['Text'].tolist()
new_encodings = tokenizer(new_texts, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
# Create a DataLoader for the new data
new_dataset = TensorDataset(new_encodings['input_ids'], new_encodings['attention_mask'])
new_loader = DataLoader(new_dataset, batch_size=batch_size, shuffle=False)

In [ ]:

# Predict labels for new data
model.eval()
all_predictions = []
with torch.no_grad():
    for batch in new_loader:
        input_ids, attention_mask = batch
        input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Get predictions
        _, predicted = torch.max(logits, dim=1)
        all_predictions.extend(predicted.cpu().numpy())

# Map numerical labels back to class names
predicted_labels = label_encoder.inverse_transform(all_predictions)

# Add predictions to the new_data DataFrame
df['label'] = predicted_labels

subset = df[['IDText', 'Text', 'label']]
subset.to_csv('subset.csv', index=False)

from google.colab import files
files.download('subset.csv')

tidak_ada_text = df[['IDText', 'label']]
tidak_ada_text.to_csv('tidak_ada_text.csv', index=False)

from google.colab import files
files.download('tidak_ada_text.csv')



CONTOH YANG BENAR

In [ ]:
from transformers import BertForSequenceClassification, AdamW
import torch.optim as optim

# Load model architecture (already done)
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', num_labels=len(labels))

# Define optimizer and learning rate
optimizer = AdamW(model.parameters(), lr=2e-5)

# Define loss function (CrossEntropyLoss for classification)
criterion = torch.nn.CrossEntropyLoss()

# Define device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training loop
epochs = 10  # Example, adjust as needed
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        total_correct = 0
        total_count = 0
        for batch in test_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Calculate accuracy
            _, predicted = torch.max(logits, dim=1)
            total_correct += (predicted == labels).sum().item()
            total_count += labels.size(0)

        accuracy = total_correct / total_count
        print(f'Epoch {epoch + 1}/{epochs}, Validation Accuracy: {accuracy:.4f}')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10, Validation Accuracy: 0.6508
Epoch 2/10, Validation Accuracy: 0.7300
Epoch 3/10, Validation Accuracy: 0.7258
Epoch 4/10, Validation Accuracy: 0.7235
Epoch 5/10, Validation Accuracy: 0.7374
Epoch 6/10, Validation Accuracy: 0.7400
Epoch 7/10, Validation Accuracy: 0.7380
Epoch 8/10, Validation Accuracy: 0.7386
Epoch 9/10, Validation Accuracy: 0.7386
Epoch 10/10, Validation Accuracy: 0.7394


In [ ]:
# Function to clean data
def clean_tweet(tweet):
    import re

    # Define regex pattern to match usernames, hyperlinks, and old style retweets
    pattern = r'(@[\w/+=]+)|(https?://\S+)|(\[RE\s+\w+\])'

    # Remove usernames, hyperlinks, and old style retweets
    tweet = re.sub(pattern, '', tweet)

    # Remove numbers
    tweet = re.sub('[0-9]+', '', tweet)

    # Remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)

    # Menghapus teks retweet gaya lama "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)

    # Remove special characters using regular expressions (re) hashtag
    tweet = re.sub(r'[^\w\s]', '', tweet)

    return tweet

# Apply the cleaning function to every row in the 'text' column of the DataFrame
df['Text'] = df['Text'].apply(clean_tweet)


In [ ]:
#import stopword
from nltk.corpus import stopwords
stopwords_indonesia = stopwords.words('indonesian')

#import sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#tokenize
from nltk.tokenize import TweetTokenizer

# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';(', ':\U0001F92D'
    ])

# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)

In [ ]:
# Remove emoji
def remove_emojis(tweet):
    """
    Removes emojis from text while preserving punctuation, special characters,
    and characters from various languages (Korean, Chinese, Russian, German, Spanish, etc.).

    Args:
        tweet (str): The text to remove emojis from.

    Returns:
        str: The text with emojis removed.
    """

    # Unicode emoji character ranges (excluding flags)
    emoji_ranges = [
        u"\U0001F600-\U0001F64F",  # emoticons
        u"\U0001F300-\U0001F5FF",  # symbols & pictographs
        u"\U0001F680-\U0001F6FF",  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"   # miscellaneous symbols
    ]

    # Combine emoji ranges into a single pattern
    emoji_pattern = "|".join(emoji_ranges)

    # Create a pattern that matches any character except letters, numbers, punctuation,
    # whitespace, and characters from various languages (excluding flags)
    allowed_chars = r"[a-zA-Z0-9\s\-.,!?:;\(\)\[\]\{\}]"
    non_emoji_pattern = f"{allowed_chars}"

    # Combine the emoji and non-emoji patterns into a single pattern
    combined_pattern = f"(^{non_emoji_pattern})|({emoji_pattern})"

    # Remove emojis while keeping other characters
    return re.sub(combined_pattern, r"\1", tweet, flags=re.UNICODE)

df['Text'] = df['Text'].apply(remove_emojis)

In [ ]:
#Remove incorrect emoji
def incorrect_emoji(tweet):
    # Define patterns for incorrect encodings and non-standard characters
    encoding_pattern = re.compile(r"Ã[A-Za-z0-9ÂŒÂˆÂŠÂœ]+")

    # Remove these patterns from the text
    tweet = re.sub(encoding_pattern, '', tweet)

    # Remove any remaining non-standard characters
    tweet = re.sub(r'[^\x00-\x7F]+', '', tweet)

    return tweet

df['Text'] = df['Text'].apply(incorrect_emoji)

In [ ]:
from indoNLP.preprocessing import replace_slang

# Apply replace_slang to the 'text' column
df['Text'] = df['Text'].apply(lambda x: replace_slang(str(x)))

# Display the first few rows of the DataFrame to verify the changes
df.head()


,IDText,Text
0,TXT0001,Lu mau orang prodemokrasi di negara ini bisa p...
1,TXT0002,Prabowo ditanya soal hutang luar negeri dia me...
2,TXT0003,kiki_daliyo Ganjar Pranowo itulah beliau soso...
3,TXT0004,Prabowo Gibran yang bisa melakukan itu semua ...
4,TXT0005,Lah justru yang enggak menyambung junjungan...


In [ ]:
from indoNLP.preprocessing import replace_word_elongation

# Apply replace_word_elongation function to the 'text' column of your DataFrame
df['Text'] = df['Text'].apply(replace_word_elongation)


In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Download stopwords for the Indonesian language
nltk.download('stopwords')
nltk.download('punkt')

# Tokenize the text
df['Text'] = df['Text'].apply(lambda x: word_tokenize(str(x).lower()))  # Convert to lowercase

# Remove stopwords from tokens
stop_words = set(stopwords.words('indonesian'))
df['Text'] = df['Text'].apply(lambda x: [word for word in x if word not in stop_words])

# Function to remove stopwords from list of words
def remove_stopwords(words):
    factory = StopWordRemoverFactory()
    stopword_remover = factory.create_stop_word_remover()
    cleaned_words = stopword_remover.remove(' '.join(words)).split()
    return cleaned_words

# Apply the remove_stopwords function to each list of words in the 'text' column
df['Text'] = df['Text'].apply(remove_stopwords)

# Join the list of words back into a string
df['Text'] = df['Text'].apply(' '.join)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Tokenize the text
df['Text'] = df['Text'].apply(lambda x: word_tokenize(str(x).lower()))  # Convert to lowercase

# Remove stopwords from tokens
stop_words = set(stopwords.words('indonesian'))
df['Text'] = df['Text'].apply(lambda x: [word for word in x if word not in stop_words])


<ipython-input-81-f18efb8b9ff0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Text'] = df['Text'].apply(lambda x: word_tokenize(str(x).lower()))  # Convert to lowercase
<ipython-input-81-f18efb8b9ff0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Text'] = df['Text'].apply(lambda x: [word for word in x if word not in stop_words])


In [ ]:
import torch
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Create the model
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', num_labels=len(label_encoder.classes_))


pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define maximum length for sequences
max_length = 128

# Tokenize the texts with max_length
train_encodings = tokenizer(list(balanced_train_df['text']), padding=True, truncation=True, max_length=max_length, return_tensors="pt")
test_encodings = tokenizer(list(test_df['text']), padding=True, truncation=True, max_length=max_length, return_tensors="pt")

# Convert labels to numerical indices
train_labels = [label_map[label] for label in balanced_train_df['label']]
test_labels = [label_map[label] for label in test_df['label']]

# Create TensorDatasets
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels))
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(test_labels))

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# # Convert text to TF-IDF features
# tfidf_vectorizer = TfidfVectorizer(max_features=5000)
# tfidf_features = tfidf_vectorizer.fit_transform(data['text']).toarray()


In [ ]:
# from imblearn.over_sampling import SMOTE

# # Split the data into training and testing sets
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(
#     tfidf_features, data['encoded_label'], test_size=0.2, stratify=data['encoded_label'], random_state=42)

# # Apply SMOTE
# smote = SMOTE(random_state=42)
# X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# # Verify the balanced class distribution
# print(pd.Series(y_resampled).value_counts())


encoded_label
5    2288
4    2288
3    2288
6    2288
7    2288
1    2288
0    2288
2    2288
Name: count, dtype: int64


# COBA-COBA

MODEL PREPARATION

In [ ]:
pip install scikit-learn

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.8, stratify=df['label'], random_state=42)


In [ ]:
# Balance the training data if it's imbalanced
labels = train_df['label'].value_counts()
max_count = labels.max()

balanced_train_dfs = []

for label in labels.index:
    label_df = train_df[train_df['label'] == label]
    if len(label_df) < max_count:
        label_df = resample(label_df, replace=True, n_samples=max_count, random_state=42)
    balanced_train_dfs.append(label_df)

balanced_train_df = pd.concat(balanced_train_dfs)



In [ ]:
from sklearn.preprocessing import LabelEncoder

# Label encoding
label_encoder = LabelEncoder()
balanced_train_df['label'] = label_encoder.fit_transform(balanced_train_df['label'])
test_df['label'] = label_encoder.transform(test_df['label'])


In [ ]:
# Ensure that the test set respects the imbalances found in the train set
balanced_train_df = train_df.groupby('label').apply(lambda x: x.sample(frac=1)).reset_index(drop=True)

# Define labels
labels = ["Ideologi", "Politik", "Ekonomi", "Sosial Budaya", "Pertahanan dan Keamanan", "Sumber Daya Alam", "Geografi", "Demografi"]
# Extract text data as a list of strings
train_texts = balanced_train_df['text'].tolist()
test_texts = test_df['text'].tolist()

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load IndoBERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', num_labels=len(labels))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# # Split the data into train and test sets (3/4 train, 1/4 test)
# train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# Ensure that the test set respects the imbalances found in the train set
balanced_train_df = train_df.groupby('label').apply(lambda x: x.sample(frac=1)).reset_index(drop=True)

# Define labels
labels = ["Ideologi", "Politik", "Ekonomi", "Sosial Budaya", "Pertahanan dan Keamanan", "Sumber Daya Alam", "Geografi", "Demografi"]

# Continue with the IndoBERT processing
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load IndoBERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', num_labels=len(labels))

# Extract text data as a list of strings
train_texts = balanced_train_df['text'].tolist()
test_texts = test_df['text'].tolist()

# Define maximum length for sequences
max_length = 128


# Tokenize the texts with max_length
train_encodings = tokenizer(train_texts, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
test_encodings = tokenizer(test_texts, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
# Define labels
labels = ["Ideologi", "Politik", "Ekonomi", "Sosial Budaya", "Pertahanan dan Keamanan", "Sumber Daya Alam", "Geografi", "Demografi"]

# Convert labels to numerical indices
label_map = {label: i for i, label in enumerate(labels)}
train_labels = balanced_train_df['label'].apply(lambda x: label_map[x]).tolist()
test_labels = test_df['label'].apply(lambda x: label_map[x]).tolist()
from torch.utils.data import TensorDataset, DataLoader

# Create TensorDatasets
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels))
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(test_labels))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install transformers==4.28.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


In [ ]:
# Create DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Calculate accuracy
            _, predicted = torch.max(logits, dim=1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

        accuracy = balanced_accuracy_score(all_labels, all_predictions)
        print(f'Epoch {epoch + 1}/{epochs}, Validation Balanced Accuracy: {accuracy:.4f}')

Epoch 1/10, Validation Balanced Accuracy: 0.2100
Epoch 2/10, Validation Balanced Accuracy: 0.3490


In [ ]:
# Join the list of words back into a string
df['text'] = df['text'].apply(' '.join)

# Split the data into train and test sets (3/4 train, 1/4 test)
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# Ensure that the test set respects the imbalances found in the train set
balanced_train_df = train_df.groupby('label').apply(lambda x: x.sample(frac=1)).reset_index(drop=True)

# Define labels
labels = ["Ideologi", "Politik", "Ekonomi", "Sosial Budaya", "Pertahanan dan Keamanan", "Sumber Daya Alam", "Geografi", "Demografi"]

# Continue with the IndoBERT processing
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load IndoBERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', num_labels=len(labels))

# Extract text data as a list of strings
train_texts = balanced_train_df['text'].tolist()
test_texts = test_df['text'].tolist()

# Define maximum length for sequences
max_length = 128


# Tokenize the texts with max_length
train_encodings = tokenizer(train_texts, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
test_encodings = tokenizer(test_texts, padding=True, truncation=True, max_length=max_length, return_tensors="pt")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Create PyTorch datasets
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TextDataset(train_encodings, train_labels)
test_dataset = TextDataset(test_encodings, test_labels)

NameError: name 'train_labels' is not defined

In [ ]:
# Define labels
labels = ["Ideologi", "Politik", "Ekonomi", "Sosial Budaya", "Pertahanan dan Keamanan", "Sumber Daya Alam", "Geografi", "Demografi"]

# Convert labels to numerical indices
label_map = {label: i for i, label in enumerate(labels)}
train_labels = balanced_train_df['label'].apply(lambda x: label_map[x]).tolist()
test_labels = test_df['label'].apply(lambda x: label_map[x]).tolist()


In [ ]:
from torch.utils.data import TensorDataset, DataLoader

# Create TensorDatasets
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels))
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(test_labels))

# Define batch size
batch_size = 16

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


<ipython-input-29-cbb8da47ce09>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels))
<ipython-input-29-cbb8da47ce09>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(test_labels))


In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import TensorDataset, DataLoader

# Define cleaning functions
def clean_tweet(tweet):
    pattern = r'(@[\w/+=]+)|(https?://\S+)|(\[RE\s+\w+\])'
    tweet = re.sub(pattern, '', tweet)
    tweet = re.sub('[0-9]+', '', tweet)
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'[^\w\s]', '', tweet)
    return tweet

def remove_emojis(tweet):
    emoji_ranges = [u"\U0001F600-\U0001F64F", u"\U0001F300-\U0001F5FF", u"\U0001F680-\U0001F6FF", u"\U0001F1E0-\U0001F1FF"]
    emoji_pattern = "|".join(emoji_ranges)
    allowed_chars = r"[a-zA-Z0-9\s\-.,!?:;\(\)\[\]\{\}]"
    combined_pattern = f"(^{allowed_chars})|({emoji_pattern})"
    return re.sub(combined_pattern, r"\1", tweet, flags=re.UNICODE)

def incorrect_emoji(tweet):
    encoding_pattern = re.compile(r"Ã[A-Za-z0-9ÂŒÂˆÂŠÂœ]+")
    tweet = re.sub(encoding_pattern, '', tweet)
    tweet = re.sub(r'[^\x00-\x7F]+', '', tweet)
    return tweet

# Load the unlabeled dataset
unlabeled_df = pd.read_csv('/content/dataset_unlabeled_penyisihan_bdc_2024.csv', sep=';')

# Apply preprocessing
unlabeled_df['text'] = unlabeled_df['text'].apply(clean_tweet)
unlabeled_df['text'] = unlabeled_df['text'].apply(remove_emojis)
unlabeled_df['text'] = unlabeled_df['text'].apply(incorrect_emoji)

stopwords_indonesia = set(stopwords.words('indonesian'))
stemmer = StemmerFactory().create_stemmer()

unlabeled_df['text'] = unlabeled_df['text'].apply(lambda x: word_tokenize(str(x).lower()))
unlabeled_df['text'] = unlabeled_df['text'].apply(lambda x: [word for word in x if word not in stopwords_indonesia])
unlabeled_df['text'] = unlabeled_df['text'].apply(' '.join)
unlabeled_df['text'] = unlabeled_df['text'].apply(lambda x: word_tokenize(str(x).lower()))
unlabeled_df['text'] = unlabeled_df['text'].apply(lambda x: [word for word in x if word not in stopwords_indonesia])

# Join the list of words back into a string
unlabeled_df['text'] = unlabeled_df['text'].apply(' '.join)

# Load IndoBERT tokenizer and model (ensure the model is loaded with the right parameters)
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', num_labels=8)

# Tokenize the texts
max_length = 128
unlabeled_texts = unlabeled_df['text'].tolist()
unlabeled_encodings = tokenizer(unlabeled_texts, padding=True, truncation=True, max_length=max_length, return_tensors="pt")

# Create TensorDataset
unlabeled_dataset = TensorDataset(unlabeled_encodings['input_ids'], unlabeled_encodings['attention_mask'])

# Create DataLoader
batch_size = 16
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=batch_size, shuffle=False)

# Move model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Prediction loop
model.eval()
all_predictions = []
with torch.no_grad():
    for batch in unlabeled_loader:
        input_ids, attention_mask = batch
        input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Get predictions
        _, predicted = torch.max(logits, dim=1)
        all_predictions.extend(predicted.cpu().numpy())

# Define the label map (make sure it matches your original training labels)
labels = ["Ideologi", "Politik", "Ekonomi", "Sosial Budaya", "Pertahanan dan Keamanan", "Sumber Daya Alam", "Geografi", "Demografi"]
label_map = {i: label for i, label in enumerate(labels)}

# Convert numerical labels back to original labels
predicted_labels = [label_map[pred] for pred in all_predictions]

# Add predictions to the DataFrame
unlabeled_df['predicted_label'] = predicted_labels

# Save the DataFrame with predictions
unlabeled_df.to_csv('/content/dataset_unlabeled_penyisihan_bdc_2024.csv', index=False, sep=';')

print("Predictions saved to 'dataset_labeled_penyisihan_bdc_2024.csv'")


KeyError: 'text'

In [ ]:
df = pd.read_csv('/content/dataset_penyisihan_bdc_2024.csv', sep=";")


In [ ]:
df.head()

,text,label
0,Kunjungan Prabowo ini untuk meresmikan dan men...,Sumber Daya Alam
1,RT Anies dapat tepuk tangan meriah saat jadi R...,Politik
2,@CIqXqwGAT04tMtx4OCATxjoVq7vv/Y8HeYaIOgMFg8Y= ...,Demografi
3,RT @L3R8XFBw3WGbxRPSj0/0hHZTbqVGX7qtfwRg9zmhK7...,Politik
4,Anies Baswedan Harap ASN termasuk TNI dan Polr...,Politik


In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Download stopwords and punkt for tokenization
nltk.download('stopwords')
nltk.download('punkt')

# Load dataset
df = pd.read_csv('/content/dataset_penyisihan_bdc_2024.csv', sep=";")

# Function to clean data
def clean_tweet(tweet):
    import re
    pattern = r'(@[\w/+=]+)|(https?://\S+)|(\[RE\s+\w+\])'
    tweet = re.sub(pattern, '', tweet)
    tweet = re.sub('[0-9]+', '', tweet)
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'[^\w\s]', '', tweet)
    return tweet

# Apply the cleaning function
df['text'] = df['text'].apply(clean_tweet)

# Define stopwords
stopwords_indonesia = stopwords.words('indonesian')

# Tokenize the text
df['text'] = df['text'].apply(lambda x: word_tokenize(str(x).lower()))

# Remove stopwords from tokens
df['text'] = df['text'].apply(lambda x: [word for word in x if word not in stopwords_indonesia])

# Create a stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Function to stem words
def stem_words(words):
    return [stemmer.stem(word) for word in words]

# Apply the stemming function to each list of words in the 'text' column
df['text'] = df['text'].apply(stem_words)

# Join the list of words back into a string
df['text'] = df['text'].apply(' '.join)

# Display the first few rows of the DataFrame to verify the changes
print(df.head())





[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


KeyboardInterrupt: 

In [ ]:
from sklearn.preprocessing import LabelEncoder  # Import LabelEncoder

# Label encoding
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Split the data into train and test sets (3/4 train, 1/4 test)
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# Ensure that the test set respects the imbalances found in the train set
balanced_train_df = train_df.groupby('label').apply(lambda x: x.sample(frac=1)).reset_index(drop=True)

# Define labels
labels = ["Ideologi", "Politik", "Ekonomi", "Sosial Budaya", "Pertahanan dan Keamanan", "Sumber Daya Alam", "Geografi", "Demografi"]

# Continue with the IndoBERT processing
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load IndoBERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', num_labels=len(labels))

# Extract text data as a list of strings
train_texts = balanced_train_df['text'].tolist()
test_texts = test_df['text'].tolist()

# Define maximum length for sequences
max_length = 128

# Tokenize the texts with max_length
train_encodings = tokenizer(train_texts, padding='max_length', truncation=True, max_length=max_length, return_tensors="pt")
test_encodings = tokenizer(test_texts, padding='max_length', truncation=True, max_length=max_length, return_tensors="pt")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Convert labels to tensors
train_labels = torch.tensor(balanced_train_df['label'].values, dtype=torch.long)
test_labels = torch.tensor(test_df['label'].values, dtype=torch.long)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [ ]:
# Create PyTorch datasets
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TextDataset(train_encodings, train_labels)
test_dataset = TextDataset(test_encodings, test_labels)

In [ ]:
!pip cache purge

Files removed: 56


In [ ]:
!conda create -n myenv python=3.7  # Create a new virtual environment (adjust Python version if needed)
!conda activate myenv  # Activate the virtual environment

!pip install transformers[torch]  # Install transformers with PyTorch dependencies
!pip install accelerate>=0.21.0  # Install accelerate with the minimum required version

import torch
from transformers import TrainingArguments, Trainer

/bin/bash: line 1: conda: command not found
/bin/bash: line 1: conda: command not found


In [ ]:
!pip install accelerate>=0.21.0
!pip install accelerate -U
!pip install transformers[torch]
import torch
from transformers import TrainingArguments, Trainer


In [ ]:
!pip show transformers

Name: transformers
Version: 4.41.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


In [ ]:
!pip uninstall -y transformers accelerate


Found existing installation: transformers 4.41.2
Uninstalling transformers-4.41.2:
  Successfully uninstalled transformers-4.41.2
Found existing installation: accelerate 0.31.0
Uninstalling accelerate-0.31.0:
  Successfully uninstalled accelerate-0.31.0


In [ ]:
!pip install transformers[torch]==4.30.0 accelerate==0.21.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1


In [ ]:
!pip show transformers accelerate


In [ ]:
# Ensure you are using the correct versions of the required packages
!pip install accelerate -U
!pip install transformers[torch] -U

In [ ]:
import torch
from transformers import TrainingArguments, Trainer, BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch"
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

TRAINING EVALUATION

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=lambda p: {'balanced_accuracy': balanced_accuracy_score(p.label_ids, p.predictions.argmax(-1))}
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()

print("Balanced Accuracy:", results['eval_balanced_accuracy'])


In [ ]:
from transformers import AdamW